In [ ]:
git clone https://github.com/ultralytics/yolov5  # clone
cd yolov5
pip install -r requirements.txt  # install

In [4]:
import os
#from os.path import join, getsize
import torch
import numpy as np
import random
from  pylabel import importer
from shutil import move

In [19]:
CWDPath = os.getcwd()
#print(CWDPath)
DataPath = CWDPath + '\\archive\\'
imagesPath = DataPath + 'images\\'
annotationsPath = DataPath + 'annotations\\'
labelsPath = DataPath + 'labels\\'
numOfImages = len(os.listdir(imagesPath))
numOfAnnotations = len(os.listdir(annotationsPath))
print("There are", numOfImages, "image files and", numOfAnnotations, "annotations files.")
print(numOfImages==numOfAnnotations)

There are 853 image files and 853 annotations files.
True


In [21]:
trainImgPath = imagesPath + 'Training\\'
trainLabelPath = labelsPath + 'Training\\'
os.makedirs(trainImgPath, exist_ok = True)
os.makedirs(trainLabelPath, exist_ok= True)
#print(trainImgPath)
validationImgPath = imagesPath + 'Validation\\'
validationLabelPath = labelsPath + 'Validation\\'
os.makedirs(validationImgPath, exist_ok = True)
os.makedirs(validationLabelPath, exist_ok= True)

testImgPath = imagesPath + 'Test\\'
testLabelPath = labelsPath + 'Test\\'
os.makedirs(testImgPath, exist_ok = True)
os.makedirs(testLabelPath, exist_ok= True)

In [22]:
OriginalLabels = importer.ImportVOC(annotationsPath)
YoloLabelsPath = OriginalLabels.export.ExportToYoloV5(trainLabelPath)

In [23]:
#Split the dataset into 6 2 2 Or 8 2 Or 8 1.8 0.2
from torch import rand


trainDataSize = int(0.8 * numOfImages)
testDataSize = numOfImages - trainDataSize
validationDataSize = int(testDataSize*0.1)

indexList = list(range(numOfImages))
random.seed(0)
random.shuffle(indexList)
trainList = indexList[:trainDataSize]
valList = indexList[trainDataSize:trainDataSize+validationDataSize]
testList = indexList[trainDataSize+validationDataSize:]


for index in trainList:
    imageName = f'maksssksksss{index}.png'
    move(imagesPath + imageName, trainImgPath + imageName)

for index in valList:
    imageName = f'maksssksksss{index}.png'
    labelName = f'maksssksksss{index}.txt'
    move(imagesPath + imageName, validationImgPath + imageName)
    move(trainLabelPath + labelName, validationLabelPath + labelName)


for index in testList:
    imageName = f'maksssksksss{index}.png'
    labelName = f'maksssksksss{index}.txt'
    move(imagesPath + imageName, testImgPath + imageName)
    move(trainLabelPath + labelName, testLabelPath + labelName)



In [24]:
l1, l2 = len(os.listdir(trainImgPath)), len(os.listdir(trainLabelPath))
print(f'trainImgNum:{l1} == trainLabelNum:{l2}\t {l1==l2}')
l3, l4 = len(os.listdir(testImgPath)), len(os.listdir(testLabelPath))
print(f'testImgNum:{l3} == testLabelNum:{l4}\t {l3==l4}')

trainImgNum:682 == trainLabelNum:682	 True
testImgNum:154 == testLabelNum:154	 True


In [26]:
classnames = []
with open(labelsPath + 'dataset.yaml', 'r') as configFile:
    for line in configFile:
        if "- " in line:
            classname = line.split()[1]
            classnames.append(classname)
print(f'classnames:{classnames}')
with open(labelsPath + 'FirstTry.yaml', 'w') as configFile:
    configFile.write(f'path: {DataPath}\n')
    configFile.write(f'train: images\\Training\n')
    configFile.write(f'val: images\\Validation\n')
    configFile.write(f'test: images\\Test\n\n')
    configFile.write(f'names:\n 0: {classnames[0]}\n 1: {classnames[1]}\n 2: {classnames[2]}')

classnames:['without_mask', 'with_mask', 'mask_weared_incorrect']


In [ ]:
python  ./yolov5/train.py --data ./archive/labels/FirstTry.yaml --weights yolov5s.pt --workers 1

In [ ]:
python ./yolov5/val.py  --data ./archive/labels/FirstTry.yaml --weights ./yolov5/runs/train/exp5/weights/best.pt

In [ ]:
python ./yolov5/detect.py --source ./archive/images/Test --weights ./yolov5/runs/train/exp5/weights/best.pt